# Import toolboxes used by the script and its functions

In [1]:
import os
import numpy as np
import pandas as pd
import re
from SPARQLWrapper import SPARQLWrapper, JSON
from unidecode import unidecode
import urllib
import json
from pandas.io.json import json_normalize
import datetime

# Set list to capture summary information for reporting at then end of the run
summary = []

# Set time run started
start = datetime.datetime.now()

# Add start time and holding space for end time to summary information
summary.append(["Processing started:" , (start.strftime('%Y-%m-%d %H:%M:%S'))]) 
summary.append(["Processing finished:" , ""])
summary.append(["" , ""])

# Add filepaths for input file here

In [2]:
inputfile = os.path.normpath('C:/Users/rthomas/Documents/GitHub/ICES2NVS_semantic_map/example/ICES2P01_test_dset.xlsx')

# Add filepaths for mapping file here

In [3]:
mapfile = os.path.normpath('C:/Users/rthomas/Documents/GitHub/ICES2NVS_semantic_map/mappings/unmapped_substances.xlsx')
biotamap = os.path.normpath('C:/Users/rthomas/Documents/GitHub/ICES2NVS_semantic_map/mappings/biota_synonym_mapping.xlsx')
p02_file = os.path.normpath('C:/Users/rthomas/Documents/GitHub/ICES2NVS_semantic_map/mappings/ICES2P02_mapping.txt')

# Generate output file name

In [4]:
fileout = os.path.normpath(inputfile[:-5]+'_mapped.xlsx')

# Define functions used by the script

In [5]:
def xstr(s):
    """Function to remove non-ASCII characters from NVS results"""
    if s is None:
        return ''
    return str(unidecode(s))

def sparql_nvs_json(s):
    """Function to input a SPARQL query (s) into the NVS SPARQL endpoint"""
    sparql = SPARQLWrapper("http://vocab.nerc.ac.uk/sparql/sparql")
    sparql.setQuery(s)
    sparql.setReturnFormat(JSON)
    r = sparql.query().convert()
    return r

def S27_map():
    """Function to get NVS:S27 chemical substrances which have CAS numbers as a published mapping"""
    s =  """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
                
            select ?codval ?prefLabel ?casurl
            where {
            <http://vocab.nerc.ac.uk/collection/S27/current/> skos:member ?url .
            ?url skos:notation ?codval .
            ?url skos:prefLabel ?prefLabel .
            ?url owl:deprecated 'false' .
            ?url owl:sameAs ?casurl .
            FILTER(regex(str(?casurl), "https://chem.nlm.nih.gov/chemidplus/rn/", "i"))
            }"""                
    r = sparql_nvs_json(s)    
    list = []
    for i in range(0,len(r['results']['bindings'])):
        a = xstr(r['results']['bindings'][i]['codval']['value'].replace('SDN:S27::',''))
        b = xstr(r['results']['bindings'][i]['prefLabel']['value'])
        c = xstr(r['results']['bindings'][i]['casurl']['value'].replace('http://chem.sis.nlm.nih.gov/chemidplus/rn/',''))
        list.append([a,b,c])
    return list

def taxon_map(spcs,aphia):
    """Function to get NVS:S25 TAXON from an AphiaID and species"""
    s =  """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
                
            select ?prefLabel
            where {
                    <http://vocab.nerc.ac.uk/collection/S25/current/> skos:member ?url .
                    ?url skos:notation ?codval .
                    ?url skos:prefLabel ?prefLabel .
                    ?url owl:deprecated 'false' .
                    FILTER(CONTAINS(?prefLabel,'%s')).
                    FILTER(CONTAINS(?prefLabel,'WoRMS %s)')).
                }""" % (spcs, aphia)
    r = sparql_nvs_json(s)
    list = []
    if len(r['results']['bindings']) == 0:
        a = 'Not available'
    else:
        label_list = []
        for i in range(0,len(r['results']['bindings'])):
            h = re.sub('\s\[.*?\]' ,'',xstr(r['results']['bindings'][i]['prefLabel']['value']))
            if h not in label_list:
                label_list.append(h)
                a = h
    list.append([a, aphia])
    return list

def S25_lookup(spcs,aphia,label):
    """Function to get NVS:S25 codval from the generated preflabel, if it exists"""
    s =  """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
                
            select ?codval ?prefLabel
            where {
            <http://vocab.nerc.ac.uk/collection/S25/current/> skos:member ?url .
            ?url skos:notation ?codval .
            ?url skos:prefLabel ?prefLabel .
            ?url owl:deprecated 'false' .
            FILTER(CONTAINS(?prefLabel,'%s (')).
            FILTER(CONTAINS(?prefLabel,'WoRMS %s')).
            FILTER(STRENDS(?prefLabel,'%s')).
            }""" % (spcs, aphia, label)
    r = sparql_nvs_json(s)
    list = []
    if len(r['results']['bindings']) == 0:
        a = 'No S25 term. Needs adding to NVS'
        b = "%s (ITIS: ?????: WoRMS %s) %s" % (spcs, aphia, label)
    else:
        for i in range(0,len(r['results']['bindings'])):
            a = xstr(r['results']['bindings'][i]['codval']['value'].replace('SDN:S25::',''))
            b = xstr(r['results']['bindings'][i]['prefLabel']['value'])
    list.append([a,b])
    return list

## Load P01 terms and semantic model vocabularies from the NERC Vocabulary Server

In [6]:
#%% Get the latest semantic model vocabulary contents from the NVS Sparql endpoint

a1 = "http://vocab.nerc.ac.uk/sparql/sparql?query=PREFIX+skos%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23%3E%0D%0A++++PREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0A++++%0D%0A++++select+%3F"
a2 = "+%3F"
a3 = "%0D%0A++++where+%7B%0D%0A++++%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2F"
a4 = "%2Fcurrent%2F%3E+skos%3Amember+%3Furl+.%0D%0A++++%3Furl+skos%3AprefLabel+%3F"
a5 = "+.%0D%0A++++%3Furl+skos%3Anotation+%3Fc+.%0D%0A++++%3Furl+owl%3Adeprecated+%27false%27+.%0D%0A++++BIND%28replace%28str%28%3Fc%29%2C%27SDN%3A"
a6 = "%3A%3A%27%2C%27%27%2C%27i%27%29+AS+%3F"
a7 = "%29%0D%0A++++%7D&output=csv&stylesheet="

S06 = pd.read_csv(a1+'S06'+a2+'S06_label'+a3+'S06'+a4+'S06_label'+a5+'S06'+a6+'S06'+a7)
S07 = pd.read_csv(a1+'S07'+a2+'S07_label'+a3+'S07'+a4+'S07_label'+a5+'S07'+a6+'S07'+a7)
S02 = pd.read_csv(a1+'S02'+a2+'S02_label'+a3+'S02'+a4+'S02_label'+a5+'S02'+a6+'S02'+a7)
S26 = pd.read_csv(a1+'S26'+a2+'S26_label'+a3+'S26'+a4+'S26_label'+a5+'S26'+a6+'S26'+a7)
P01 = pd.read_csv(a1+'P01'+a2+'P01_label'+a3+'P01'+a4+'P01_label'+a5+'P01'+a6+'P01'+a7)

# Download semantic component mapping

urlS06 = "http://vocab.nerc.ac.uk/sparql/sparql?query=PREFIX+skos%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23%3E%0D%0APREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0A%0D%0Aselect+distinct+%3FS06+%3FP01%0D%0Awhere+%7B%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FS06%2Fcurrent%2F%3E+skos%3Amember+%3Furla+.%0D%0A%3Furla+owl%3Adeprecated+%27false%27+.%0D%0A%3Furla+skos%3Anotation+%3Fn2+.%0D%0A%3Furla+skos%3Anarrower+%3Furlb+.%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FP01%2Fcurrent%2F%3E+skos%3Amember+%3Furlb+.%0D%0A%3Furlb+owl%3Adeprecated+%27false%27+.%0D%0A%3Furlb+skos%3Anotation+%3Fn1+.%0D%0ABIND%28replace%28%3Fn1%2C+%22SDN%3AP01%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FP01%29+.%0D%0ABIND%28replace%28%3Fn2%2C+%22SDN%3AS06%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FS06%29+.%0D%0A%7D&output=csv&stylesheet="
S06_P01 = pd.read_csv(urlS06)
print("S06 mapping downloaded.")

urlS07 = "http://vocab.nerc.ac.uk/sparql/sparql?query=PREFIX+skos%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23%3E%0D%0APREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0A%0D%0Aselect+distinct+%3FS07+%3FP01%0D%0Awhere+%7B%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FS07%2Fcurrent%2F%3E+skos%3Amember+%3Furla+.%0D%0A%3Furla+owl%3Adeprecated+%27false%27+.%0D%0A%3Furla+skos%3Anotation+%3Fn2+.%0D%0A%3Furla+skos%3Anarrower+%3Furlb+.%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FP01%2Fcurrent%2F%3E+skos%3Amember+%3Furlb+.%0D%0A%3Furlb+owl%3Adeprecated+%27false%27+.%0D%0A%3Furlb+skos%3Anotation+%3Fn1+.%0D%0ABIND%28replace%28%3Fn1%2C+%22SDN%3AP01%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FP01%29+.%0D%0ABIND%28replace%28%3Fn2%2C+%22SDN%3AS07%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FS07%29+.%0D%0A%7D&output=csv&stylesheet="
S07_P01 = pd.read_csv(urlS07)
print("S07 mapping downloaded.")

urlS27 = "http://vocab.nerc.ac.uk/sparql/sparql?query=PREFIX+skos%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23%3E%0D%0APREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0A%0D%0Aselect+distinct+%3FS27+%3FP01%0D%0Awhere+%7B%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FS27%2Fcurrent%2F%3E+skos%3Amember+%3Furla+.%0D%0A%3Furla+owl%3Adeprecated+%27false%27+.%0D%0A%3Furla+skos%3Anotation+%3Fn2+.%0D%0A%3Furla+skos%3Anarrower+%3Furlb+.%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FP01%2Fcurrent%2F%3E+skos%3Amember+%3Furlb+.%0D%0A%3Furlb+owl%3Adeprecated+%27false%27+.%0D%0A%3Furlb+skos%3Anotation+%3Fn1+.%0D%0ABIND%28replace%28%3Fn1%2C+%22SDN%3AP01%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FP01%29+.%0D%0ABIND%28replace%28%3Fn2%2C+%22SDN%3AS27%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FS27%29+.%0D%0A%7D&output=csv&stylesheet="
S27_P01 = pd.read_csv(urlS27)
print("S27 mapping downloaded.")

urlS02 = "http://vocab.nerc.ac.uk/sparql/sparql?query=PREFIX+skos%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23%3E%0D%0APREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0A%0D%0Aselect+distinct+%3FS02+%3FP01%0D%0Awhere+%7B%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FS02%2Fcurrent%2F%3E+skos%3Amember+%3Furla+.%0D%0A%3Furla+owl%3Adeprecated+%27false%27+.%0D%0A%3Furla+skos%3Anotation+%3Fn2+.%0D%0A%3Furla+skos%3Arelated+%3Furlb+.%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FP01%2Fcurrent%2F%3E+skos%3Amember+%3Furlb+.%0D%0A%3Furlb+owl%3Adeprecated+%27false%27+.%0D%0A%3Furlb+skos%3Anotation+%3Fn1+.%0D%0ABIND%28replace%28%3Fn1%2C+%22SDN%3AP01%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FP01%29+.%0D%0ABIND%28replace%28%3Fn2%2C+%22SDN%3AS02%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FS02%29+.%0D%0A%7D&output=csv&stylesheet="
S02_P01 = pd.read_csv(urlS02)
print("S02 mapping downloaded.")

urlS26 = "http://vocab.nerc.ac.uk/sparql/sparql?query=PREFIX+skos%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23%3E%0D%0APREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0A%0D%0Aselect+distinct+%3FS26+%3FP01%0D%0Awhere+%7B%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FS26%2Fcurrent%2F%3E+skos%3Amember+%3Furla+.%0D%0A%3Furla+owl%3Adeprecated+%27false%27+.%0D%0A%3Furla+skos%3Anotation+%3Fn2+.%0D%0A%3Furla+skos%3Anarrower+%3Furlb+.%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FP01%2Fcurrent%2F%3E+skos%3Amember+%3Furlb+.%0D%0A%3Furlb+owl%3Adeprecated+%27false%27+.%0D%0A%3Furlb+skos%3Anotation+%3Fn1+.%0D%0ABIND%28replace%28%3Fn1%2C+%22SDN%3AP01%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FP01%29+.%0D%0ABIND%28replace%28%3Fn2%2C+%22SDN%3AS26%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FS26%29+.%0D%0A%7D&output=csv&stylesheet="
S26_P01 = pd.read_csv(urlS26)
print("S26 mapping downloaded.")

urlS25 = "http://vocab.nerc.ac.uk/sparql/sparql?query=PREFIX+skos%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23%3E%0D%0APREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0A%0D%0Aselect+distinct+%3FS25+%3FP01%0D%0Awhere+%7B%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FS25%2Fcurrent%2F%3E+skos%3Amember+%3Furla+.%0D%0A%3Furla+owl%3Adeprecated+%27false%27+.%0D%0A%3Furla+skos%3Anotation+%3Fn2+.%0D%0A%3Furla+skos%3Anarrower+%3Furlb+.%0D%0A%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FP01%2Fcurrent%2F%3E+skos%3Amember+%3Furlb+.%0D%0A%3Furlb+owl%3Adeprecated+%27false%27+.%0D%0A%3Furlb+skos%3Anotation+%3Fn1+.%0D%0ABIND%28replace%28%3Fn1%2C+%22SDN%3AP01%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FP01%29+.%0D%0ABIND%28replace%28%3Fn2%2C+%22SDN%3AS25%3A%3A%22%2C+%22%22%2C+%22i%22%29+AS+%3FS25%29+.%0D%0A%7D&output=csv&stylesheet="
S25_P01 = pd.read_csv(urlS25)
print("S25 mapping downloaded.")

# Build P01 semantic model dataframe
P01 = pd.merge(P01, S06_P01, how='left', on='P01')
P01 = pd.merge(P01, S07_P01, how='left', on='P01')
P01 = pd.merge(P01, S27_P01, how='left', on='P01')
P01 = pd.merge(P01, S02_P01, how='left', on='P01')
P01 = pd.merge(P01, S26_P01, how='left', on='P01')
P01 = pd.merge(P01, S25_P01, how='left', on='P01')

P01 = P01.fillna(value={'S25': 'BE007736', 'S07': 'S0700006'})

display(P01.head())

S06 mapping downloaded.
S07 mapping downloaded.
S27 mapping downloaded.
S02 mapping downloaded.
S26 mapping downloaded.
S25 mapping downloaded.


,P01,P01_label,S06,S07,S27,S02,S26,S25
0,PRESPR01,Pressure (spatial coordinate) exerted by the w...,S0600168,S0700006,NaN,S023,MAT00640,BE007736
1,DOXMZZXX,Concentration of oxygen {O2 CAS 7782-44-7} per...,S0600045,S0700006,CS002779,S057,MAT00633,BE007736
2,DOXYMMOP,Concentration recalculated to zero salinity of...,S0600056,S0700006,CS002779,S053,MAT00633,BE007736
3,DOXYAAOP,Concentration of oxygen {O2 CAS 7782-44-7} per...,S0600045,S0700006,CS002779,S053,MAT00633,BE007736
4,OXYTAAOP,Temperature of oxygen determination by optode,S0600082,S0700006,NaN,NaN,NaN,BE007736


## Load ICES semantic model components for mapping to P01 semantic model from file into a Pandas DataFrame

In [9]:
inputs = pd.read_excel(inputfile)
param_combo = inputs.copy(deep=True)

# Insert number of rows in the input file in to the summary information
print("Rows input: %s" % len(inputs))
summary.append(["Rows input:", len(inputs)])

# Make a working copy of the parameter combinations for mapping and remove duplicate combinations
param_combo = inputs.copy(deep=True).drop_duplicates(keep=False)
input_duplicates = len(inputs) - len(param_combo)
print("Rows duplicated: %s" % input_duplicates)
summary.append(["Rows duplicated:", input_duplicates])
summary.append(["" , ""])
print("Rows for mapping: %s" % len(param_combo))
summary.append(["Rows for mapping:", len(param_combo)])




# Add columns needed for P01 semantic model
# In the working copy set NaNs to '-9' and add columns for mapped NVS semantic model elements
param_combo = param_combo.fillna('-9')
param_combo = param_combo.assign(S06_label='',              # Measurement Property
                                 S07_label='not specified', # Measurement Property Statistic
                                 S02_label='',              # Measurement - Matrix relationship
                                 )
param_combo['PARAM'] = param_combo['PARAM'].str.upper()
param_combo['AphiaID'] = param_combo['AphiaID'].astype('int32')
# Remove leading or trailing spaces from the text columns
columns = param_combo.columns.tolist()
columns.remove('AphiaID')
for column in columns:
    param_combo[column] = param_combo[column].str.strip()

Rows input: 5439
Rows duplicated: 44
Rows for mapping: 5395


# Mapping of chemical PARAMs and NVS S27 vocabulary entries
### Determine where direct mappings already published on the NVS or available in a local file

In [11]:
# First get S27 terms that have a mapping to ICES PARAM vocabulary published from the NVS
q = """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
                    
    select ?PARAM ?S27 ?S27_label 
    where {
           <http://vocab.nerc.ac.uk/collection/S27/current/> skos:member ?url .
           ?url skos:notation ?a .
           ?url skos:prefLabel ?S27_label .
           ?url owl:deprecated 'false' .
           ?url skos:related ?c .
           FILTER(regex(str(?c), "http://vocab.ices.dk/services/rdf/collection/PARAM/", "i")) .
           BIND(substr(?a,10,8) as ?S27) .
           BIND(replace(str(?c), "http://vocab.ices.dk/services/rdf/collection/PARAM/", "", "i") AS ?PARAM) .
          }"""
# URL for the above query is:
url = """http://vocab.nerc.ac.uk/sparql/sparql?query=PREFIX+skos%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23%3E%0D%0A++++PREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0A++++++++++++++++++++%0D%0A++++select+%3FPARAM+%3FS27+%3FS27_label+%0D%0A++++where+%7B%0D%0A+++++++++++%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FS27%2Fcurrent%2F%3E+skos%3Amember+%3Furl+.%0D%0A+++++++++++%3Furl+skos%3Anotation+%3Fa+.%0D%0A+++++++++++%3Furl+skos%3AprefLabel+%3FS27_label+.%0D%0A+++++++++++%3Furl+owl%3Adeprecated+%27false%27+.%0D%0A+++++++++++%3Furl+skos%3Arelated+%3Fc+.%0D%0A+++++++++++FILTER%28regex%28str%28%3Fc%29%2C+%22http%3A%2F%2Fvocab.ices.dk%2Fservices%2Frdf%2Fcollection%2FPARAM%2F%22%2C+%22i%22%29%29+.%0D%0A+++++++++++BIND%28substr%28%3Fa%2C10%2C8%29+as+%3FS27%29+.%0D%0A+++++++++++BIND%28replace%28str%28%3Fc%29%2C+%22http%3A%2F%2Fvocab.ices.dk%2Fservices%2Frdf%2Fcollection%2FPARAM%2F%22%2C+%22%22%2C+%22i%22%29+AS+%3FPARAM%29+.%0D%0A++++++++++%7D&output=CSV&stylesheet="""

# More efficient to ingest SPARQL response as a CSV directly into a Pandas DataFrame
mapped_chems = pd.read_csv(url)
mapped_chems['SOURCE'] = 'NVS'

# Get local ICES PARAM to NVS S27 substance mapping from mapping file location provided earlier
local_map = pd.read_excel(mapfile)
local_map['SOURCE'] = mapfile

# Combine NVS mappings with those from the local file to generate a complete list of known mappings
full_chem_map = pd.concat([mapped_chems, local_map[['PARAM','S27','S27_label','SOURCE']]]).drop_duplicates(subset='PARAM').reset_index(drop=True)

# Identify any mappings in the local file already published from the NVS
duplicate_map = pd.merge(mapped_chems, local_map, how='inner', on='PARAM')


print("Number of ICES PARAM terms mapped to S27 chemical substance terms from NVS: %s" % (len(mapped_chems)))
summary.append(["Number of ICES PARAM terms mapped to S27 chemical substance terms from NVS:", (len(mapped_chems))])

print("Number of local chemical substance mappings from file: %s" % (len(local_map)))
summary.append(["Number of local chemical substance mappings from file:" , (len(local_map))])

print("Number of chemical substance mappings in local file and NVS: %s" % (len(duplicate_map)))
summary.append(["Number of chemical substance mappings in local file and NVS:" , (len(duplicate_map))])

Number of ICES PARAM terms mapped to S27 chemical substance terms from NVS: 0
Number of local chemical substance mappings from file: 43
Number of chemical substance mappings in local file and NVS: 0


### For PARAMs with CAS numbers check if the chemical substance exists within S27 and can be mapped via CAS number

In [13]:
# SPARQL query for all NVS substances with CAS numbers from the SPARQL endpoint
q =  """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
            
    select ?nvs_codval ?nvs_prefLabel ?nvs_casrn
    where {
           <http://vocab.nerc.ac.uk/collection/S27/current/> skos:member ?url .
           ?url skos:notation ?a .
           ?url skos:prefLabel ?nvs_prefLabel .
           ?url owl:deprecated 'false' .
           ?url owl:sameAs ?c .
           FILTER(regex(str(?c), "https://chem.nlm.nih.gov/chemidplus/rn/", "i")) .
           BIND(replace(str(?a),'SDN:S27::','','i') AS ?nvs_codval) .
           BIND(replace(str(?c),'https://chem.nlm.nih.gov/chemidplus/rn/','','i') AS ?nvs_casrn) .
          }"""                

# URL for the above query is:
url = "http://vocab.nerc.ac.uk/sparql/sparql?query=PREFIX%20skos%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23%3E%0APREFIX%20owl%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0A%0Aselect%20%3Fnvs_codval%20%3Fnvs_prefLabel%20%3Fnvs_casrn%0Awhere%20%7B%0A%20%20%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FS27%2Fcurrent%2F%3E%20skos%3Amember%20%3Furl%20.%0A%20%20%3Furl%20skos%3Anotation%20%3Fa%20.%0A%20%20%3Furl%20skos%3AprefLabel%20%3Fnvs_prefLabel%20.%0A%20%20%3Furl%20owl%3Adeprecated%20'false'%20.%0A%20%20%3Furl%20owl%3AsameAs%20%3Fc%20.%0A%20%20FILTER(regex(str(%3Fc)%2C'https%3A%2F%2Fchem.nlm.nih.gov%2Fchemidplus%2Frn%2F'%2C'i'))%20.%0A%20%20BIND(replace(str(%3Fa)%2C'SDN%3AS27%3A%3A'%2C''%2C'i')%20AS%20%3Fnvs_codval)%20.%0A%20%20BIND(replace(str(%3Fc)%2C'https%3A%2F%2Fchem.nlm.nih.gov%2Fchemidplus%2Frn%2F'%2C''%2C'i')%20AS%20%3Fnvs_casrn)%20.%0A%7D&output=csv&stylesheet="   

# More efficient to ingest SPARQL response as a CSV directly into a Pandas DataFrame
nvs_cas = pd.read_csv(url)

display(nvs_cas)

,nvs_codval,nvs_prefLabel,nvs_casrn
0,CS002531,hexachlorobenzene,118-74-1
1,CS002244,"4,4'-dichlorodiphenyldichloroethane",72-54-8
2,CS026903,silicate,17181-37-2
3,CS002447,cobalt,7440-48-4
4,CS000151,total phosphorus,7723-14-0
5,CS001117,"2,4'-dichlorodiphenyltrichloroethane",789-02-6
6,CS001103,"2,4'-dichlorodiphenyldichloroethane",53-19-0
7,CS002877,nitrite,14797-65-0
8,CS026904,phosphate,14265-44-2
9,CS002896,chlorophyll-a,479-61-8


### For the contents of the input file determine if any new PARAM to S27 mappings are required

In [15]:
# Get subset of chemical PARAMS in the input list
input_PARAMS = param_combo[['PARAM','PRNAM','CAS']][param_combo['CAS']!='-9'].drop_duplicates().reset_index(drop=True)

print("Number of PARAM chemical substances from input file for P01 mapping: %s" % (len(input_PARAMS)))
summary.append(["Number of PARAM chemical substances from input file for P01 mapping:" , (len(input_PARAMS))])

# Merge information to determine which mappings are missing from the NVS
compare = pd.merge(input_PARAMS, full_chem_map, how='outer', on='PARAM')
S27_missing = compare[['PARAM','CAS']][compare['S27'].isnull()].copy(deep=True)

print("Number of PARAM chemical substances in file not directly mapped: %s" % (len(S27_missing)))
summary.append(["Number of PARAM chemical substances in file unable to be mapped:" , (len(S27_missing))])

# Merge information between unmapped PARAMs and NVS S27 based on CAS number
S27_casmap_review = pd.merge(S27_missing, nvs_cas, how='inner', left_on='CAS', right_on='nvs_casrn')

print("Number of potential PARAM-S27 mappings via CAS linkage: %s for %s PARAMs" % (len(S27_casmap_review), len(S27_casmap_review['PARAM'].drop_duplicates())))
summary.append(["    Number of potential PARAM-S27 mappings for review based on CAS linkage:", "%s for %s PARAMs" % (len(S27_casmap_review), len(S27_casmap_review['PARAM'].drop_duplicates()))])

print("Number of PARAMs with no S27 term identified via CAS number: %s" % (len(pd.concat([S27_missing['PARAM'], S27_casmap_review['PARAM']]).drop_duplicates(keep=False))))
summary.append(["    Number of PARAMs with no S27 term identified via CAS number:" , "%s" % len(pd.concat([S27_missing['PARAM'], S27_casmap_review['PARAM']]).drop_duplicates(keep=False))])
display(pd.concat([S27_missing['PARAM'], S27_casmap_review['PARAM']]).drop_duplicates(keep=False))

Number of PARAM chemical substances from input file for P01 mapping: 260
Number of PARAM chemical substances in file not directly mapped: 219
Number of potential PARAM-S27 mappings via CAS linkage: 215 for 198 PARAMs
Number of PARAMs with no S27 term identified using CAS: 21


41         TBTIN
45         TPTIN
52         BBJKF
59         BDE66
86         CHRTR
105        OCDAN
118         BBKF
246    CB138+163
247      CB56+60
248        CDFDX
249         CDFO
250        PEBDE
251         SCB7
252        DDTEP
253        HPBDE
254        HXBDE
255         NBDE
256         OBDE
257        TEBDE
258        TRBDE
259        PFOSA
Name: PARAM, dtype: object

### Where mappings don't exist for chemical PARAMs or can't be made via CAS number as listed above:
### Either:
### - Add mapping to local file and re-run previous steps
### - Continue process and these rows will not be able to be mapped

In [16]:
# Add holding text for PARAM to S27 mappings from CAS for review
if len(S27_casmap_review)>0:
    warn = pd.DataFrame()
    warn['PARAM'] = S27_casmap_review['PARAM'].drop_duplicates().copy(deep=True).reset_index(drop=True)
    warn['S27'] = 'Potential S27 term exists.'
    warn['S27_label'] = 'Mapping needs to be reviewed and added to NVS or unmapped_substance.xlsx.'
    warn['SOURCE'] = 'CAS linkage exists.'
    
    full_chem_map = pd.concat([full_chem_map, warn])
    
# Add holding text for PARAM where no potential S27 exists through CAS
if len(S27_missing)>0:
    warn = pd.DataFrame()
    warn['PARAM'] = pd.concat([S27_missing['PARAM'], S27_casmap_review['PARAM']]).drop_duplicates(keep=False).copy(deep=True).reset_index(drop=True)
    warn['S27'] = 'No S27 term identified from CAS.'
    warn['S27_label'] = 'Potential new term to be added to S27 and mapped to PARAM.'
    warn['SOURCE'] = 'No CAS linkage found.'
    
    full_chem_map = pd.concat([full_chem_map, warn])

# Add S27 mappings to the main table    
param_combo = pd.merge(param_combo, full_chem_map[['PARAM','S27','S27_label']], how='left', on='PARAM')

# Mark rows that do not require mapping as "Not applicable."    
param_combo = param_combo.fillna(value={'S27': 'not applicable', 'S27_label': 'not applicable'})

print("Total combinations = %s" % (len(param_combo)))
PARAMs2map = pd.DataFrame()
PARAMs2map = param_combo[['PARAM','PRNAM']][param_combo['S27']=='not applicable'].drop_duplicates()

print("PARAMs with no mapping for review: %s" % len(PARAMs2map))

display(PARAMs2map)

Total combinations = 5395
PARAMs with no mapping for review: 31


,PARAM,PRNAM
38,CORG,organic carbon
122,CS137,cesium-137
140,GSMF1000,Grain Size Mass Fraction <1000 micron
141,GSMF125,Grain Size Mass Fraction <125 micron
142,GSMF2000,Grain Size Mass Fraction <2000 micron
143,GSMF63,Grain Size Mass Fraction <63 micron (silt/clay)
156,LOIGN,loss on ignition
157,MOCON%,moisture content percent
161,NORG,organic nitrogen (N)
704,SUSP,suspended solids


## Mapping of DTYPE, MATRX and METPT combinations to S26

In [17]:
matrix_check = param_combo[['DTYPE','MATRX','METPT']].drop_duplicates().copy(deep=True).reset_index(drop=True)

# Set to default of 'Check MATRX. Not mapped.'
matrix_check = matrix_check.assign(S26_label = 'Check MATRX. Not mapped.')

print("Number of MATRX for P01 mapping: %s" % len(matrix_check))
summary.append(["Number of MATRX for P01 mapping:" , len(matrix_check)])

for index, row in matrix_check.iterrows():
    if row['DTYPE'] == 'CF':
        # Set S26 label to 'biota'
        row['S26_label'] = 'biota'

    elif row['DTYPE'] == 'CS':
        if row['MATRX'] == 'SEDTOT':
            row['S26_label'] = 'sediment'
        elif row['MATRX'][3:len(row['MATRX'])] != 'TOT':
            row['S26_label'] = 'sediment <'+row['MATRX'][3:len(row['MATRX'])] +'um'
        else:
            row['S26_label'] = 'Check MATRX'
            
    elif row['DTYPE'] == 'CW':
        if row['MATRX'] == 'WT':
            if row['METPT'] == '-9':
                row['S26_label'] = 'water body [dissolved plus reactive particulate <unknown phase]'
            else:
                metpt_list = row['METPT'].split('~')               
                for metpt in metpt_list:
                    if metpt in ('NF','NONE','NA','CP'):
                        row['S26_label'] = 'water body [dissolved plus reactive particulate phase]'
                        continue
                    elif metpt in('GFF','GF/F','FF-GF-0.7'):
                        row['S26_label'] = 'water body [dissolved plus reactive particulate <GF/F phase]'
                        continue
                    elif metpt in('GFC','GF/C','FF-GF-1.2','FF-PP-1.2'):
                        row['S26_label'] = 'water body [dissolved plus reactive particulate <GF/C phase]'
                        continue
                    elif metpt in('FM-PC-0.4','FM-PC-0.45','FM-PES-0.45','FM-CN-0.45','FM-CA-0.45','PCF40','PCF45','PCF'):
                        row['S26_label'] = 'water body [dissolved plus reactive particulate <0.4/0.45um phase]'
                        continue
                    elif metpt in('F'):
                        row['S26_label'] = 'water body [dissolved plus reactive particulate <unknown phase]'
                        continue
                    elif metpt in('FM-CA-0.2'):
                        row['S26_label'] = 'water body [dissolved plus reactive particulate <0.2um phase]'
                        continue
                    else:
                        if row['S26_label'] == '':
                            row['S26_label'] = 'Check METPT'
    #print("Row %s of %s matrix combinations mapped." % (index+1,len(matrix_check))) # Commented out used in debugging
#display(matrix_check)    
# Subset potential S26 new entries
S26new = matrix_check[matrix_check['S26_label']=='Check MATRX. Not mapped.']

print("Number of potential new S26 terms: %s" % len(S26new))
summary.append(["Number of potential new S26 terms:" , len(S26new)])
display(S26new)

# Add S26 semantic model mapping to the main table based on the combinations provided   
param_combo = pd.merge(param_combo, matrix_check, how='left', on=['DTYPE','MATRX','METPT'])

print("Total combinations = %s" % (len(param_combo)))

Number of MATRX for P01 mapping: 64
Number of potential new S26 terms: 5


,DTYPE,MATRX,METPT,S26_label
8,CW,SPM,NONE,Check MATRX. Not mapped.
9,CW,SPM,FM-PC-0.4,Check MATRX. Not mapped.
10,CW,SPM,NF,Check MATRX. Not mapped.
11,CW,SPM,FF-GF-1.2,Check MATRX. Not mapped.
12,CW,SPM,GFC1.2,Check MATRX. Not mapped.


Total combinations = 5395


# Taxon, WoRMS AphiaID, ITIS TSN combination check

In [18]:
# Get all existing TAXONs from S25 and simplify text labels to show distinct TAXON values from the S25 semantic model
url = """http://vocab.nerc.ac.uk/sparql/sparql?query=PREFIX+skos%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23%3E%0D%0A++++PREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0A++++++++++++++++++++%0D%0A++++select+distinct+%3FAphiaID+%3FTAXON%0D%0A++++where+%7B%0D%0A+++++++++++%3Chttp%3A%2F%2Fvocab.nerc.ac.uk%2Fcollection%2FS25%2Fcurrent%2F%3E+skos%3Amember+%3Furl+.%0D%0A+++++++++++%3Furl+skos%3AprefLabel+%3FprefLabel+.%0D%0A+++++++++++%3Furl+owl%3Adeprecated+%27false%27+.%0D%0A+++++++++++FILTER%28regex%28str%28%3FprefLabel%29%2C+%22WoRMS%22%2C+%22i%22%29%29+.%0D%0A+++++++++++BIND%28replace%28str%28%3FprefLabel%29%2C+%22%5C%5C+%5C%5C%5B.*%3F%5C%5C%5D%22%2C%22%22%2C+%22i%22%29+AS+%3FTAXON%29+.%0D%0A+++++++++++BIND%28replace%28replace%28replace%28str%28%3FTAXON%29%2C+%22%5C%5C%29%22%2C%22%22%2C+%22i%22%29%2C+%22.*%28%3F%3DWoRMS+%29%22%2C+%22%22%2C+%22i%22%29%2C+%22WoRMS+%22%2C+%22%22%2C+%22i%22%29+AS+%3FAphiaID%29+.%0D%0A++++++++++%7D%0D%0A++++order+by+%3FAphiaID%0D%0A&output=CSV&stylesheet=CSV"""

S25taxon = pd.read_csv(url)

# Identify multiple TAXONs per AphiaID within S25
S25taxon_duplicates = S25taxon[S25taxon.duplicated(['AphiaID'], keep=False)].copy(deep=True)
S25taxon_duplicates.replace(u'\xc2\xa0',u' ', regex=True, inplace=True)
S25taxon_duplicates.replace(u'\u2019',u"'", regex=True, inplace=True)

# Remove duplicate TAXON records from S25taxon dataframe
S25taxon_clean = pd.concat([S25taxon, S25taxon_duplicates]).drop_duplicates(keep=False).copy(deep=True).reset_index(drop=True)

# Create a Pandas DataFrame and populate with unique combinations of Species and AphiaID from the input file
input_taxa_check = pd.DataFrame()
input_taxa_check = param_combo[['Species','AphiaID']][param_combo['Species']!='-9'].drop_duplicates().reset_index(drop=True)
input_taxa_check = input_taxa_check.astype({"AphiaID": int})

print("Number of Species for P01 mapping: %s" % len(input_taxa_check))
summary.append(["Number of Species for P01 mapping:" , len(input_taxa_check)])
#%%
# Function to call WoRMS web service
def worms_check(url):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    if response.code==204:
        e = 'No AphiaID found.'
    elif response.code==206:
        e = 'Multiple AphiaID found.'
    elif response.code==200:
        e = response.read()
    return e

# If AphiaID is absent then lookup using the WoRMS web service
for index, row in input_taxa_check.iterrows():
    if row['AphiaID'] == -9:
        if '&' not in row['Species']:
            url = 'http://marinespecies.org/rest/AphiaIDByName/%s?marine_only=true' % row['Species'].replace(" ","%20")
            input_taxa_check.loc[index, 'AphiaID'] = worms_check(url)
        else:
            input_taxa_check.loc[index, 'AphiaID'] = 'Combination of taxa'

inputs_aphia = pd.merge(inputs, input_taxa_check, on='Species')


#%% 
# Get WoRMS scientific names from AphiaID provided using WoRMS web service
def worms_check(url):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    if response.code==204:
        list.append('No response.')
    elif response.code==200:
        e = response.read()
    return json.loads(e)
       
worms = pd.DataFrame()        

aphia_list = input_taxa_check['AphiaID'].tolist()
y = len(aphia_list)
if y<50:
    ids = ''
    for i in range(0,50):
        ids = ids + 'aphiaids%5B%5D=' + str(aphia_list[i]) + '&'
    url = 'http://www.marinespecies.org/rest/AphiaRecordsByAphiaIDs?%s' % ids[0:-1]
    worms = pd.DataFrame(worms_check(url), ignore_index=True)
elif y>50:
    for j in range(0,int(y/50)):
        ids = ''
        for i in range(j*50,min((j+1)*50,y)):
            ids = ids + 'aphiaids%5B%5D=' + str(aphia_list[i]) + '&'
        url = 'http://www.marinespecies.org/rest/AphiaRecordsByAphiaIDs?%s' % ids[0:-1]
        worms = pd.concat([worms, pd.DataFrame(worms_check(url))], ignore_index=True)
    ids = ''
    for i in range((j+1)*50,min((j+2)*50,y)):
        ids = ids + 'aphiaids%5B%5D=' + str(aphia_list[i]) + '&'
    url = 'http://www.marinespecies.org/rest/AphiaRecordsByAphiaIDs?%s' % ids[0:-1]
    worms = pd.concat([worms, pd.DataFrame(worms_check(url))], ignore_index=True)

input_taxa_check = pd.merge(input_taxa_check, worms[['AphiaID','scientificname']], how='left', on='AphiaID')

input_taxa_check = input_taxa_check.rename(index=str, columns={'scientificname': 'name_from_AphiaID'})

# Set column to indicate if a discrepancy to be resolved exists based on Scientific names not matching
a = input_taxa_check.Species == input_taxa_check.name_from_AphiaID
input_taxa_check['proceed'] = np.where(a, 'Yes', 'No')

# Subset those taxa where naming discrepancy exists
taxa_discrepancy = input_taxa_check[input_taxa_check['proceed']=='No'].reset_index(drop=True)

print("Number of Species with name discrepancy: %s" % len(taxa_discrepancy))
summary.append(["Number of Species with name discrepancy:" , len(taxa_discrepancy)])
display(taxa_discrepancy)

# Map AphiaID to S25 component TAXON for the non-duplicate AphiaID results in S25
taxa_map = pd.merge(input_taxa_check[['AphiaID','name_from_AphiaID']],
                    S25taxon_clean, 
                    how='left', 
                    on='AphiaID')
taxa_map = taxa_map.fillna(value={'TAXON': 'New TAXON required.'}).drop_duplicates()


# Add TAXON mapping to the main table based on the AphiaID provided    
param_combo = pd.merge(param_combo, taxa_map, how='left', on='AphiaID')

# Mark rows that do not require mapping as "Not applicable."    
param_combo = param_combo.fillna(value={'TAXON': 'not specified', 'name_from_AphiaID': '-9'})

print("Total combinations = %s" % (len(param_combo)))

Number of Species for P01 mapping: 116
Number of Species with name discrepancy: 9


,Species,AphiaID,name_from_AphiaID,proceed
0,Venerupis philippinarum,231750,Ruditapes philippinarum,No
1,Psetta maxima,127149,Scophthalmus maximus,No
2,Phoca hispida,159021,Pusa hispida,No
3,Mysidacea,149668,Mysida,No
4,Raja radiata,105865,Amblyraja radiata,No
5,Clupea harengus membras,126417,Clupea harengus,No
6,Clupea harengus membras,126417,Clupea harengus,No
7,Sebastes marinus,151324,Sebastes norvegicus,No
8,Macoma baltica,880017,Limecola balthica,No


Total combinations = 5395


# Iterate through file for ICES combinations where a mapping does not already exist.

In [ ]:
for row in range(0,len(param_combo)):
    if param_combo.loc[row,['P01_Code']][0]=='-9':

        # Logic for contaminants in sediment mappings
        if param_combo.loc[row,['DTYPE']][0] == 'CS':
            param_combo.loc[row,['S25']] = 'BE007736'
            param_combo.loc[row,['S25_label']] = 'not applicable'
            if param_combo.loc[row,['MUNIT']][0] == '%':
                param_combo.loc[row,['S06_label']] = "Proportion"
            elif param_combo.loc[row,['MUNIT']][0][len(param_combo.loc[row,['MUNIT']][0])-1] == 'g':
                param_combo.loc[row,['S06_label']] = 'Concentration'
            else:
                param_combo.loc[row,['S06_label']] = 'Check MUNIT'
                
            if param_combo.loc[row,['BASIS']][0] == 'D':
                param_combo.loc[row,['S02_label']] = 'per unit dry weight of'
            elif param_combo.loc[row,['BASIS']][0] == 'W':
                param_combo.loc[row,['S02_label']] = 'per unit wet weight of'
            else:
                param_combo.loc[row,['S02_label']] = 'Check BASIS'
                
            if param_combo.loc[row,['MATRX']][0] == 'SEDTOT':
                param_combo.loc[row,['S26_label']] = 'sediment'
            elif param_combo.loc[row,['MATRX']][0][3:len(param_combo.loc[row,['MATRX']])] != 'TOT':
                param_combo.loc[row,['S26_label']] = 'sediment <'+param_combo.loc[row,['MATRX']][0][3:len(param_combo.loc[row]['MATRX'])] +'um'
            else:
                param_combo.loc[row,['S26_label']] = 'Check MATRX'

        # Logic for contaminants in water mappings       
        if param_combo.loc[row,['DTYPE']][0] == 'CW':
            param_combo.loc[row,['S25']] = 'BE007736'
            param_combo.loc[row,['S25_label']] = 'not applicable'
            if param_combo.loc[row,['MUNIT']][0] =='ntu':
                if param_combo.loc[row,['PARAM']][0]=='TURB':
                    param_combo.loc[row,['S06_label']] = 'Turbidity'
                    param_combo.loc[row,['S02_label']] = 'of the'        
            elif param_combo.loc[row,['MUNIT']][0] == '%':
                param_combo.loc[row,['S06_label']] = "Proportion"
            elif param_combo.loc[row,['MUNIT']][0] == 'mBq/l':
                param_combo.loc[row,['S06_label']] = "Activity"        
                param_combo.loc[row,['S02_label']] = 'per unit volume of the'
            elif param_combo.loc[row,['MUNIT']][0][len(param_combo.loc[row,['MUNIT']][0])-1] == 'g':
                param_combo.loc[row,['S06_label']] = 'Concentration'
                param_combo.loc[row,['S02_label']] = 'per unit mass of the'
            elif param_combo.loc[row,['MUNIT']][0][len(param_combo.loc[row,['MUNIT']][0])-1] == 'l':
                param_combo.loc[row,['S06_label']] = 'Concentration'
                param_combo.loc[row,['S02_label']] = 'per unit volume of the'
            else:
                param_combo.loc[row,['S06_label']] = 'Check MUNIT'
                param_combo.loc[row,['S02_label']] = 'Check MUNIT'
                        
            if param_combo.loc[row,['MATRX']][0] == 'WT':
                if param_combo.loc[row,['METPT']][0]=='-9':
                    param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate <unknown phase]'
                else:
                    metpt_list = param_combo.loc[row,['METPT']][0].split('~')               
                    for metpt in metpt_list:
                        if metpt in ('NF','NONE','NA','CP'):
                            param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate phase]'
                            continue
                        elif metpt in('GFF','GF/F','FF-GF-0.7'):
                            param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate <GF/F phase]'
                            continue
                        elif metpt in('GFC','GF/C','FF-GF-1.2','FF-PP-1.2'):
                            param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate <GF/C phase]'
                            continue
                        elif metpt in('FM-PC-0.4','FM-PC-0.45','FM-PES-0.45','FM-CN-0.45','FM-CA-0.45','PCF40','PCF45','PCF'):
                            param_combo.loc[row,['S26_label']] = 'water body [dissolved plus reactive particulate <0.4/0.45um phase]'
                            continue
                        elif metpt in('F'):
                            param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate <unknown phase]'
                            continue
                        elif metpt in('FM-CA-0.2'):
                            param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate <0.2um phase]'
                            continue
                        else:
                            if param_combo.loc[row,['S26_label']][0] == '':
                                param_combo.loc[row,['S26_label']]='Check METPT'

        # Logic for contaminants in biota mappings
        if param_combo.loc[row,['DTYPE']][0] == 'CF':
            # Set S26 label to 'biota'
            param_combo.loc[row,['S26_label']] = 'biota'
            
            if param_combo.loc[row,['CAS']][0] in ('-9','NA'):
                if param_combo.loc[row,['PARAM']][0]!='CS137':
                    param_combo.loc[row,['S06_label']] = 'Generate mapping'
                else:
                    param_combo.loc[row,['S06_label']] = 'Concentration'
            else:
                param_combo.loc[row,['S06_label']] = 'Concentration'

            # Map BASIS column for dry weight, wet weight and lipid normalisaed concentrations. Anything else requires checking.  
            if param_combo.loc[row,['BASIS']][0] == 'D':
                param_combo.loc[row,['S02_label']] = 'per unit dry weight of'
            elif param_combo.loc[row,['BASIS']][0] == 'W':
                param_combo.loc[row,['S02_label']] = 'per unit wet weight of'
            elif param_combo.loc[row,['BASIS']][0] == 'L':
                param_combo.loc[row,['S02_label']] = 'in'
                param_combo.loc[row,['S06_label']] = 'Lipid-normalised concentration'
            else:
                param_combo.loc[row,['S02_label']] = 'Check BASIS'
            
            # Map matrix of the biota to appropriate NVS S25 SUBCOMPONENT and/or STAGE. Not some constraints based on taxa type applied in the code.
            if param_combo.loc[row,['MATRX']][0] == 'WO':
                param_combo.loc[row,['SUBCOMPONENT']] = 'not specified'
            elif param_combo.loc[row,['MATRX']][0] == 'TM':
                param_combo.loc[row,['SUBCOMPONENT']] = 'muscle tissue'
            elif param_combo.loc[row,['MATRX']][0] == 'SI':
                param_combo.loc[row,['SUBCOMPONENT']] = 'not specified'
            elif param_combo.loc[row,['MATRX']][0] == 'SH':
                param_combo.loc[row,['SUBCOMPONENT']] = 'shell'
            elif param_combo.loc[row,['MATRX']][0] == 'SB':
                if param_combo.loc[row,['Species']][0] not in ('Gobius','Crangon crangon','Mysidacea'):
                    param_combo.loc[row,['SUBCOMPONENT']] = 'flesh'
                else:
                    param_combo.loc[row,['SUBCOMPONENT']] = 'Checking species-matrx combo validity with ICES.'
            elif param_combo.loc[row,['MATRX']][0] == 'RO':
                param_combo.loc[row,['STAGE']] = 'eggs'
                param_combo.loc[row,['SUBCOMPONENT']] = 'not specified'
            elif param_combo.loc[row,['MATRX']][0] == 'MU&EP':
                param_combo.loc[row,['SUBCOMPONENT']] = 'muscle tissues and skin'
            elif param_combo.loc[row,['MATRX']][0] == 'MU':
                if param_combo.loc[row,['Species']][0] == 'Loligo vulgaris':
                    param_combo.loc[row,['SUBCOMPONENT']] = 'flesh'
                else:
                    param_combo.loc[row,['SUBCOMPONENT']] = 'muscle tissue'
            elif param_combo.loc[row,['MATRX']][0] == 'LI':
                param_combo.loc[row,['SUBCOMPONENT']] = 'liver'
            elif param_combo.loc[row,['MATRX']][0] == 'KI':
                param_combo.loc[row,['SUBCOMPONENT']] = 'kidney'
            elif param_combo.loc[row,['MATRX']][0] == 'GO':
                param_combo.loc[row,['SUBCOMPONENT']] = 'gonads'
            elif param_combo.loc[row,['MATRX']][0] == 'GI':
                param_combo.loc[row,['SUBCOMPONENT']] = 'gill'
            elif param_combo.loc[row,['MATRX']][0] == 'FE':
                param_combo.loc[row,['SUBCOMPONENT']] = 'feathers'
            elif param_combo.loc[row,['MATRX']][0] == 'FA':
                param_combo.loc[row,['SUBCOMPONENT']] = 'body fat'
            elif param_combo.loc[row,['MATRX']][0] == 'EX':
                if param_combo.loc[row,['Species']][0] == 'Mytilus edulis':
                    param_combo.loc[row,['SUBCOMPONENT']] = 'shell'
                else:
                    param_combo.loc[row,['SUBCOMPONENT']] = 'Checking species-matrx combo validity with ICES.'
            elif param_combo.loc[row,['MATRX']][0] == 'EP':
                param_combo.loc[row,['SUBCOMPONENT']] = 'skin'
            elif param_combo.loc[row,['MATRX']][0] == 'EH':
                param_combo.loc[row,['STAGE']] = 'eggs'
                param_combo.loc[row,['SUBCOMPONENT']] = 'egg yolk and albumen homogenate'
            elif param_combo.loc[row,['MATRX']][0] == 'EG':
                param_combo.loc[row,['STAGE']] = 'eggs'
            elif param_combo.loc[row,['MATRX']][0] == 'BS':
                param_combo.loc[row,['SUBCOMPONENT']] = 'blood serum'
            elif param_combo.loc[row,['MATRX']][0] == 'BR':
                param_combo.loc[row,['SUBCOMPONENT']] = 'brain'
            elif param_combo.loc[row,['MATRX']][0] == 'BL':
                param_combo.loc[row,['SUBCOMPONENT']] = 'blood'
            elif param_combo.loc[row,['MATRX']][0] == 'BC':
                param_combo.loc[row,['SUBCOMPONENT']] = 'blood cells'
            elif param_combo.loc[row,['MATRX']][0] == 'BB':
                if param_combo.loc[row,['Note']][0] != 'Fish':
                    param_combo.loc[row,['SUBCOMPONENT']] = 'blubber'
                else:
                    param_combo.loc[row,['SUBCOMPONENT']] = 'Checking species-matrx combo validity with ICES.'

            # Where there is no Species - AphiaID discrepancy check the NVS to see if taxa is already published.
            if param_combo.loc[row,['Species']][0] in clean_taxa:
                spcs = param_combo.loc[row,['Species']][0]
                aphia = int(param_combo.loc[row,['AphiaID']][0])
                
                if len(taxon_code[taxon_code['AphiaID']==aphia])==0:
                    taxon_list = taxon_map(spcs, aphia)
                    taxon_code = pd.concat([taxon_code,pd.DataFrame.from_records(taxon_list,columns=['taxon_preflabel','AphiaID'])])
                    param_combo.loc[row,['TAXON']] = taxon_list[0][0]
                else:
                    param_combo.loc[row,['TAXON']] = taxon_code[taxon_code['AphiaID']==aphia].iloc[0]['taxon_preflabel']
                
                #Build S25 preflabel components for text matching
                label=''
                txn = param_combo.loc[row,['TAXON']][0]
                scp = param_combo.loc[row,['SUBCOMPONENT']][0]
                stg = param_combo.loc[row,['STAGE']][0]
                if np.any([txn == 'Not available', scp in ('Checking species-matrx combo validity with ICES.','New term needed.'), stg in ('Checking species-matrx combo validity with ICES.','New term needed.')]):
                    label = 'No term for at least one S25 model list. Needs adding to NVS'
                elif np.all([txn != 'Not available', stg == 'not specified', scp == 'not specified']):
                    label = ')'
                elif np.all([txn != 'Not available', stg == 'not specified', scp != 'not specified']):
                    label = '[Subcomponent: %s]' % (scp)
                elif np.all([txn != 'Not available', stg != 'not specified', scp == 'not specified']):
                    label = '[Stage: %s]' % (stg)
                else:
                    label = '[Stage: %s Subcomponent: %s]' % (stg, scp)
                param_combo.loc[row,['S25_label']] = (txn + " " + label).replace(") )", ")")
                
                # Look up S25 preflabel to identify appropriate S25 codval
                if label !='No term for at least one S25 model list. Needs adding to NVS':
                    if len(S25_code[S25_code['S25_preflabel']==label])==0:
                        S25_list = S25_lookup(spcs,aphia,label)
                        S25_code = pd.concat([S25_code,pd.DataFrame.from_records(S25_list,columns=['S25_codval','S25_preflabel'])])
                        param_combo.loc[row,['S25']] = S25_list[0][0]
                    else:
                        param_combo.loc[row,['S25']] = S25_code[S25_code['S25_preflabel']==label].iloc[0]['S25_codval']
                else:
                    param_combo.loc[row,['S25']] = label
            # Where there is a Species - AphiaID discrepancy continue with the next step.    
            elif param_combo.loc[row,['Species']][0] in query_taxa:
                param_combo.loc[row,['TAXON']] = 'To be resolved.'
        
        # Logic for mapping to ICES PARAM based on CAS registry number or unmapped file        
        cas = param_combo.loc[row,['CAS']][0]
        param = param_combo.loc[row,['PRNAM']][0]
        if param in local_map_terms:
            codval = local_map['NVS_CODVAL'][local_map['PARAM']==param]
            preflabel = local_map['NVS_PREFLABEL'][local_map['PARAM']==param]
            param_combo.loc[row,['S27']] = codval.iloc[0]
            param_combo.loc[row,['S27_label']] = preflabel.iloc[0]            
        elif cas in ('-9','NA'):
            param_combo.loc[row,['S27']] = 'not applicable'
            param_combo.loc[row,['S27_label']] = 'not applicable' 
        else:
            codval = nvs['nvs_codval'][nvs['nvs_casrn']==cas]
            preflabel = nvs['nvs_preflabel'][nvs['nvs_casrn']==cas]
            if len(nvs[nvs['nvs_casrn']==cas]) == 0:
                param_combo.loc[row,['S27']] = 'No S27 term. Needs adding to NVS'   
            elif len(nvs[nvs['nvs_casrn']==cas]) == 1:
                param_combo.loc[row,['S27']] = codval.iloc[0]
                param_combo.loc[row,['S27_label']] = preflabel.iloc[0]
            elif len(nvs[nvs['nvs_casrn']==cas]) > 1:
                if len(nvs[nvs['nvs_preflabel']==param]) == 1:
                    codval = nvs['nvs_codval'][nvs['nvs_preflabel']==param]
                    preflabel = nvs['nvs_preflabel'][nvs['nvs_preflabel']==param]
                    param_combo.loc[row,['S27']] = codval.iloc[0]
                    param_combo.loc[row,['S27_label']] = preflabel.iloc[0]
                elif len(nvs[nvs['nvs_preflabel']==param]) == 0:
                    if param == 'mercury':
                        param = 'total mercury'
                        codval = nvs['nvs_codval'][nvs['nvs_preflabel']==param]
                        preflabel = nvs['nvs_preflabel'][nvs['nvs_preflabel']==param]
                        param_combo.loc[row,['S27']] = codval.iloc[0]
                        param_combo.loc[row,['S27_label']] = preflabel.iloc[0]
    print("Row %s of %s complete." % (row+1,len(param_combo)))

# Replaces any problematic text characters from the NVS imported into the DataFrame
# that will cause issues when writing the output to file.
param_combo.replace(u'\xa0',u' ', regex=True, inplace=True)
param_combo.replace(u'\u2019',u"'", regex=True, inplace=True)

# Reorder columns for output
param_combo = param_combo[['PARGROUP','PRNAM','CAS','DTYPE','PARAM','MUNIT','MATRX','BASIS','METPT','METOA','Note','AphiaID','Species',
                          'P01_Code','S06_label','S07_label','S27_label','S27','S02_label','S26_label','S25','S25_label',
                          'TAXON','STAGE','GENDER','SIZE','SUBCOMPONENT','MORPHOLOGY','COLOUR','SUBGROUP']]

print("Total combinations in file = %s" % (len(param_combo)))
         

# Save the results of the ICES to NVS semantic model mapping to file.

In [ ]:
# Split out those combinations that have already been mapped to P01 in the parameter set
outputP01 = param_combo[param_combo['P01_Code']!='-9']

# Retain those combinations that have not yet been mapped to P01
output = param_combo[param_combo['P01_Code']=='-9']

# Split out SPM combinations to file where more worked needed at ICES to allow accurate mapping.
outputSPM = output[output['MATRX']=='SPM']

# Split out combinations that exclude SPM measurements 
output = output[output['MATRX']!='SPM']

# Generate list of new substances to be added to S27
S27_cols = ['PARGROUP','PARAM','PRNAM','CAS','S27_label','S27']
output_S27 = output[S27_cols][output['S27']=='No S27 term. Needs adding to NVS'].drop_duplicates().reset_index(drop=True)

# Generate list of new biological entities to be added to S25
S25_cols = ['Note','AphiaID','Species','MATRX','S25','S25_label','TAXON','STAGE','GENDER','SIZE','SUBCOMPONENT','MORPHOLOGY','COLOUR','SUBGROUP']
output_S25a = output[S25_cols][output['S25'] == 'No S25 term. Needs adding to NVS'].drop_duplicates().reset_index(drop=True)
output_S25b = output[S25_cols][output['S25'] == 'No term for at least one S25 model list. Needs adding to NVS'].drop_duplicates().reset_index(drop=True)
    
# Generate list of paramters with full set of semantic model terms for P01 matching script
output_complete = output[output['S25']!='No S25 term. Needs adding to NVS']
output_complete = output_complete[output_complete['TAXON']!='To be resolved.']
output_complete = output_complete[output_complete['S25']!='No term for at least one S25 model list. Needs adding to NVS']
output_complete = output_complete[output_complete['S27']!='No S27 term. Needs adding to NVS']
output_complete = output_complete[output_complete['S27']!='not applicable']

output_nonchem = output[output['S27']=='not applicable'].reset_index(drop=True)

taxa_discrepancy = taxa_check[taxa_check['proceed']=='No'].reset_index(drop=True)

# Create summary information
summary = pd.DataFrame([["Processing started:" , (start.strftime('%Y-%m-%d %H:%M:%S'))],
["Processing finished:" , datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
["" , ""],
["Rows input:", len(inputs)],
["" , ""],
["Rows with P01 provided:" , len(outputP01)],
["Rows successfully mapped:" , len(output_complete)],
["Rows unable to be mapped:" , ""],
["Because:" , ""],
["    Rows for SPM:" , len(outputSPM)],
["    Rows for non-chemical codes:" , len(output_nonchem)],
["    New chemical substances for mapping:" , len(output_S27)],
["    New biological entities for creation:" , len(output_S25a)],
["    New biological entitity semantic components:" , len(output_S25b)],
["    Taxa discrepancies:" , len(taxa_discrepancy)]])


# Save outputs as worksheets within Excel file
with pd.ExcelWriter(fileout) as writer:
    summary.to_excel(writer, sheet_name='summary', header=False, index=False)
    inputs.to_excel(writer, sheet_name='input', index=False)
    outputP01.to_excel(writer, sheet_name='known_matched', index=False)
    output_complete.to_excel(writer, sheet_name='mapped', index=False)
    outputSPM.to_excel(writer, sheet_name='SPM_codes', index=False)
    output_nonchem.to_excel(writer, sheet_name='nonchemical_codes', index=False)
    output_S27.to_excel(writer, sheet_name='new_S27', index=False)
    output_S25a.to_excel(writer, sheet_name='new_S25', index=False)
    output_S25b.to_excel(writer, sheet_name='new_S25_component', index=False)
    taxa_discrepancy.to_excel(writer, sheet_name='taxa_discrepancies', index=False)
    

In [ ]:
display(summary)